In [1]:
# %pip install ipyleaflet requests transformers torch

In [2]:
import requests
import torch
from IPython.display import display
from ipyleaflet import Map as LeafletMap, DrawControl
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

/Users/sophiakim/miniconda3/envs/virtual_env/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/Users/sophiakim/miniconda3/envs/virtual_env/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feed

In [3]:
# Once data has been processed, we check if there are any invalid data (i.e. marker is on land)
def is_valid_data(data):
    hourly_data = data['hourly']

    for hourly_entry in hourly_data.items():
        values = hourly_entry[1]
        if all(value is None for value in values):
            return False

    return True

In [4]:
# Initialize cpu for model to run on
device = torch.device("cpu")

model_name = "gpt2"

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Initialize model
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Initialize another model to refine output from gpt2
model_paraphrase_name = "facebook/bart-large-cnn"
paraphraser = pipeline("summarization", model=model_paraphrase_name)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [5]:
# Convert data into prompt
def create_prompt(data):
    prompt = "Tell me if the wind wave is strong or weak today based on this data: \n\n"
    prompt += str(data)

    return prompt

In [6]:
# Refine gpt2 output with summarization model
def clean_up_output(summary):
    refined_text = paraphraser(summary, max_length=100)
    return refined_text[0]['summary_text']

In [7]:
def retrieve_summary(prompt):
    input = tokenizer(prompt, return_tensors="pt", padding="longest").to(device)

    # move input to device
    input = {key: value.to(device) for key, value in input.items()}

    # adding pad token ID
    model.config.pad_token_id = model.config.eos_token_id
    
    output = model.generate(
        input['input_ids'],
        attention_mask=input['attention_mask'],
        max_new_tokens=150,
        num_return_sequences=1,
        no_repeat_ngram_size=2
        )
    result = tokenizer.decode(output[0], skip_special_tokens=True)

    # clean up output
    prompt_length = len(prompt) + 2
    result_without_prompt = result[prompt_length:]
    return result_without_prompt.strip()

In [8]:
# Event handler when user pins marker on map
def handle_marker(event, action, geo_json):
    coordinates = geo_json['geometry']['coordinates']
    longitude, latitude = str(coordinates[0]), str(coordinates[1])

    # Fetch sea conditions from Open-Meteo API
    url = "https://marine-api.open-meteo.com/v1/marine?latitude=" + latitude + "&longitude=" + longitude + "&hourly=wind_wave_height,wind_wave_direction,wind_wave_period&timezone=auto&forecast_days=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if is_valid_data(data):
            prompt = create_prompt(data)
            summary = retrieve_summary(prompt)
            refined_summary = clean_up_output(summary)
            print(refined_summary)
        else:
            print("You can't cruise on land!")
            return
    else:
        print("Error fetching data from Open-Meteo API. Status code: " + str(response.status_code))
    
    with open("marine_forecast.txt", "w") as file:
        file.write(refined_summary)
    print("\nForecast written to marine_forecast.txt")

In [9]:
map_center = [20.0, 0.0]
m = LeafletMap(center=map_center, zoom=2, scroll_wheel_zoom=True)

draw_control = DrawControl(
    marker={'shapeOptions': {}},
    polyline={},
    polygon={},
    circle={},
    circlemarker={},
)

m.add_control(draw_control)
draw_control.on_draw(handle_marker)

display(m)

Map(center=[20.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…